## Generating fixtures to test the Equation Of Time functions, the Longitude Fitting Functions and the Longitude Calculation Functions

### Expected Longitude Output is downloaded directly from https://maps.nrel.gov/pvdaq/ PVDAQ Contributed Sites

### Expected longitude = -76.6636

In [1]:
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
# PVInsight Code Imports
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data
path = Path.cwd().parent.parent.parent
os.chdir(path)

### Load data table from external source

#### For today's example, we're loading data from NREL's PVDAQ API, which is a publically available PV generatation data set.

In [2]:
data_frame = get_pvdaq_data(sysid=1199, year=[2015, 2016, 2017], api_key='DEMO_KEY')

CAUTION: Multiple scan rates detected!=======================] 100.0% ...queries complete in 25.0 seconds       
Scan rates (in seconds): [300, 280]
0 transitions detected.
Suggest splitting data set between:




In [3]:
gmt_offset = -5

### Running the  DataHandler

In [4]:
dh = DataHandler(data_frame[0])
dh.fix_dst()

In [5]:
dh.run_pipeline(power_col='ac_power', fix_shifts=False, correct_tz=False)

total time: 7.11 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              2.52s
Cleaning                   0.15s
Filtering/Summarizing      4.45s
    Data quality           0.19s
    Clear day detect       0.14s
    Clipping detect        3.49s
    Capacity change detect 0.63s



### Calculate the daily solar noon

In [6]:
from solardatatools.algorithms import SunriseSunset

In [7]:
ss = SunriseSunset()
ss.run_optimizer(data=dh.filled_data_matrix)

In [8]:
sunrise = ss.sunrise_measurements
sunset = ss.sunset_measurements
sunrise[np.isnan(sunrise)] = 0
sunset[np.isnan(sunset)] = 0
solarnoon = np.nanmean([sunrise, sunset], axis=0)

### Equation of time prosposed by Duffie And Rosa

In [9]:
path = Path.cwd()
os.chdir(path)

In [25]:
day_of_year = dh.day_index.dayofyear.to_numpy()
days = dh.daily_flags.no_errors

In [18]:
# Exporting input for eot algorithms
np.savetxt('./tests/fixtures/longitude/eot_input.csv', day_of_year,  delimiter=",")

In [19]:
from pvsystemprofiler.utilities.equation_of_time import eot_da_rosa, eot_duffie

eot_duffie = eot_duffie(day_of_year)
eot_da_rosa = eot_da_rosa(day_of_year)

In [20]:
# Exporting output for eot algorithms
np.savetxt('./tests/fixtures/longitude/eot_duffie_output.csv', eot_duffie,  delimiter=",")
np.savetxt('./tests/fixtures/longitude/eot_da_rosa_output.csv', eot_da_rosa,  delimiter=",")

In [26]:
# exporting inputs for fit_longitude and calculate_longitude
# solarnoon
np.savetxt('./tests/fixtures/longitude/solarnoon.csv', solarnoon,  delimiter=",")
# days
np.savetxt('./tests/fixtures/longitude/days.csv', days,  delimiter=",")

### Fit longitude

In [29]:
from pvsystemprofiler.algorithms.longitude.fitting import fit_longitude

fit_longitude(eot_duffie, solarnoon, days, gmt_offset, loss='l2')

-77.22534574490635

### Calculate Longitude

In [28]:
from pvsystemprofiler.algorithms.longitude.calculation import calculate_longitude

calculate_longitude(eot_duffie, solarnoon, days, gmt_offset)

-77.10636729272031